In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# list all files
data_path = 'data/'
for dirPath, dirNames, fileNames in os.walk(data_path):
    print(dirPath, dirNames, fileNames)

df_train = pd.read_csv(data_path + fileNames[1], header=None)
train_Y = pd.read_csv(data_path + fileNames[2], header=None)
df_test = pd.read_csv(data_path + fileNames[0], header=None)
# train_Y.columns = ['label']
# df_train = pd.concat([df_train, train_Y], axis=1)
# df_train.head()

data/ [] ['test.csv', 'train.csv', 'trainLabels.csv']


In [3]:
df_train.dtypes

0     float64
1     float64
2     float64
3     float64
4     float64
5     float64
6     float64
7     float64
8     float64
9     float64
10    float64
11    float64
12    float64
13    float64
14    float64
15    float64
16    float64
17    float64
18    float64
19    float64
20    float64
21    float64
22    float64
23    float64
24    float64
25    float64
26    float64
27    float64
28    float64
29    float64
30    float64
31    float64
32    float64
33    float64
34    float64
35    float64
36    float64
37    float64
38    float64
39    float64
dtype: object

In [4]:
# GradientBoostingClassifier
x_train, x_val, y_train, y_val = train_test_split(df_train, train_Y, test_size=0.1, random_state=10)
clf = GradientBoostingClassifier(random_state=7)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_val)

C:\Users\wavegis\anaconda3\envs\yienv\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
acc = metrics.accuracy_score(y_pred, y_val)
print(f'accuracy: {acc}')

accuracy: 0.87


In [6]:
# KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_val)

C:\Users\wavegis\anaconda3\envs\yienv\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [7]:
acc = metrics.accuracy_score(y_pred, y_val)
print(f'accuracy: {acc}')

accuracy: 0.98


In [8]:
# 設定要訓練的超參數組合
clf = KNeighborsClassifier()
neighbors = np.arange(1, 20)
param_grid = dict(n_neighbors=neighbors)
grid_search = GridSearchCV(clf, param_grid, scoring='accuracy', n_jobs=-1)
grid_result = grid_search.fit(x_train, y_train)

C:\Users\wavegis\anaconda3\envs\yienv\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [9]:
grid_result.best_params_

{'n_neighbors': 6}

In [10]:
print('best accuracy: %f using %s' % (grid_result.best_score_, grid_result.best_params_))

best accuracy: 0.895556 using {'n_neighbors': 6}


In [17]:
clf_best = KNeighborsClassifier(n_neighbors=grid_result.best_params_['n_neighbors'])
clf.fit(x_train, y_train)
y_pred = clf.predict(x_val)
acc = metrics.accuracy_score(y_pred, y_val)
print(f'accuracy: {acc}')

accuracy: 0.94


C:\Users\wavegis\anaconda3\envs\yienv\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [18]:
df_test.isnull().any()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
dtype: bool

In [21]:
test_pred = pd.DataFrame(clf.predict(df_test)).reset_index()
test_pred.columns = ['Id', 'Solution']
test_pred['Id'] += 1
test_pred

,Id,Solution
0,1,1
1,2,0
2,3,1
3,4,0
4,5,0
...,...,...
8995,8996,1
8996,8997,1
8997,8998,1
8998,8999,0


In [22]:
test_pred.to_csv('submission.csv', index=False)

In [23]:
submission = pd.read_csv('submission.csv')
submission

,Id,Solution
0,1,1
1,2,0
2,3,1
3,4,0
4,5,0
...,...,...
8995,8996,1
8996,8997,1
8997,8998,1
8998,8999,0
